In [1]:
import pandas as pd, pickle, re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np

In [2]:
from termcolor import colored

In [3]:
from IPython.core.display import HTML
#display(HTML(df2.to_html()))

In [4]:
from tensorflow.keras.models import model_from_json 
json_file = open("model.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
eda=pd.read_csv('freq.csv')

In [6]:
eda=eda[eda['text']!='친구']

In [7]:
eda=eda[eda['compare']>=2]

In [8]:
dic={'Noun': '명사',
 'Josa': '조사',
 'Adjective': '형용사',
 'Adverb': '부사',
 'Determiner': '한정사',
 'Verb': '동사',
 'Modifier': '수식어',
 'Conjunction': '접속사',
 'VerbPrefix': '동사 접두사',
 'Suffix': '접미사',
 'Exclamation': '감탄사',
 'Alpha': 'Alpha',
 'Foreign': 'Foreign',
 'Hashtag': 'Hashtag',
 'Eomi': 'Eomi',
 'Email': 'Email',
 'ScreenName': 'ScreenName'}

In [9]:
cleaning =lambda s: re.sub("[^가-힣a-zA-Z.!?\\s]","",s)
tokenizer = Okt()

with open('word_dict.pickle', 'rb') as handle:
    data = pickle.load(handle)

keras_tokenizer, max_len = data

In [10]:
def predict(a):

    list_dot = ['.'*i for i in range(20, 1, -1)]
    model = loaded_model
    
    a = a.replace(' ','')
    
    #말줄임표 없애기
    for i in list_dot:
        a = a.replace(i,'말줄임표')

    _input1 = [tokenizer.morphs(cleaning(str(a)))]
        
    _input2 = keras_tokenizer.texts_to_sequences(_input1) 
    _input2 = pad_sequences(_input2, maxlen=max_len, padding='post')
    
    pred = model.predict(_input2)

        
    result = str(int(pred*100)) + '%'
        
    return result

In [15]:
while True:
    a = input('\n끄적끄적 해보세요: \n\n')
    if a=='':
        break
    else:
        text=[]
        content=[]
        freq=[]
        compare=[]
        a = a.replace(' ','')
        for i in tokenizer.pos(cleaning(str(a))):
            try:
                if (i[0]+'/'+dic[i[1]]) in list(eda['content']):
                    content.append(i[0]+'/'+dic[i[1]])

            except:
                continue

        content=list(set(content))
        for i in content:
            freq.append(int(eda['freq'][eda['content']==i]))
            compare.append(np.round(float(eda['compare'][eda['content']==i]),1))
            text.append(eda['freq'][eda['content']==i])

        df=pd.DataFrame({'text':text,'content':content,'freq':freq,'compare':compare}).sort_values('compare',ascending=False)
        df=df[:7]
        result=predict(a)


        if len(freq)==0 :

            print(colored('\n당신의 우울증 확률: {0} \n', attrs=['reverse', 'blink']) .format(result))
        elif int(result[:-1])<=50:

            print(colored('\n당신의 우울증 확률: {0} \n', attrs=['reverse', 'blink']) .format(result))

        else:

            print(colored('\n당신의 우울증 확률: {0} \n', attrs=['reverse', 'blink']) .format(result))
            for i in tokenizer.pos(cleaning(str(a))):
                try:
                    if (i[0]+'/'+dic[i[1]]) in content:
                        print(colored(i[0],'red', attrs=['reverse', 'blink']),end=' ')
                    else:
                        print(i[0],end=' ')
                except:
                    continue
            print('\n우울증 문서에 자주 나타나는 표현:\n')

            for i in list(df.index):
                try:
                    if df.compare[i]!=0:
                        print(df.content[i].replace('/',' / '))
                        print(': 일반문서의 '.rjust(25),str(df.compare[i]),'배 등장\n')
                    else:
                        print(df.content[i].replace('/',' / '),': 우울증 문서에만 등장')
                except:
                    continue
                    


끄적끄적 해보세요: 

행복하다

당신의 우울증 확률: 73% 


끄적끄적 해보세요: 

'현명한포기에는용기가필요하다.실패를인정하는용기.노력과시간이아무런결실을맺지못했더라도과감히버릴줄아는용기.실패했음에도새로운것에다시도전할수있는용기.현명한포기는끝까지버티다어쩔수없이하는체념이나힘들면그냥포기해버리는의지박약과는다르다.적절한시기에아직더가볼수있음에도용기를내어그만두는것이다.왜?그렇게하는것이이익이니까.인생에도손절매가필요하다.타이밍을놓치면작은손해에서그칠일이큰손해로이어진다.무작정버티고노력하는것만이능사가아니다.지금우리에겐노력보다용기가더필요한것같다.무모하지만도전하는용기그리고적절한시기에포기할줄아는용기말이다.'

당신의 우울증 확률: 57% 

현명한 포기 에는 용기 가 필요하다 실패 를 인정 하 는 용기 노력 과 시간 이 아무런 결실 을 맺지못 했 더라도 과 감히 버릴줄아는 용기 실패했음에도 새로운것에다 시도 전 할수있는 용기 현명한 포기 는 끝 까지 버티다 어쩔 수없이 하는 체념 이나 힘들면 그냥 포기 해 버리는 의지 박약 과는 다르다 적절한시기에 아직 더 가볼수있음에도 용기 를 내 어그 만 두는것이다 왜 그렇게 하는것이 이익 이니까 인생 에도 손절 매가 필요하다 타이밍 을 놓치면 작은 손해 에서 그 칠일이 큰손 해로 이어진다 무작정 버티고 노력 하는것만이 능사 가 아니다 지금 우리 에겐 노력 보다 용기 가더 필요한것 같다 무모하지만도 전하 는 용기 그리고 적절한시기에 포기 할줄아는 용기말이다 
우울증 문서에 자주 나타나는 표현:

그냥 / 명사
                 : 일반문서의  4.1 배 등장

버티다 / 동사
                 : 일반문서의  2.8 배 등장

왜 / 명사
                 : 일반문서의  2.2 배 등장

같다 / 형용사
                 : 일반문서의  2.1 배 등장


끄적끄적 해보세요: 




In [92]:
int(result[:-1])

48

In [98]:
a.replace(' ','')

'현명한포기에는용기가필요하다.실패를인정하는용기.노력과시간이아무런결실을맺지못했더라도과감히버릴줄아는용기.실패했음에도새로운것에다시도전할수있는용기.현명한포기는끝까지버티다어쩔수없이하는체념이나힘들면그냥포기해버리는의지박약과는다르다.적절한시기에아직더가볼수있음에도용기를내어그만두는것이다.왜?그렇게하는것이이익이니까.인생에도손절매가필요하다.타이밍을놓치면작은손해에서그칠일이큰손해로이어진다.무작정버티고노력하는것만이능사가아니다.지금우리에겐노력보다용기가더필요한것같다.무모하지만도전하는용기그리고적절한시기에포기할줄아는용기말이다.'